# AlbatrosDigital API client demo

This notebook is intended to provide a quick demo of the functionality provided by the Shipyard API.
It will provide examples of all endpoints and should be used as a tutorial.

The API is available at `https://api.albatros.digital/graphql`.

As per graphQL specs, queries to the api should be using the `http POST` method.
For complete documentation, there is a Graphiql interface is available at the same endpoint via a `http GET` request, or simply navigating to that address with a browser.


# Imports and initiation

In [41]:
import sys
import os

# first, make sure that the src directory is in the python path
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append(nb_dir)

from pprint import pprint

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd


# initialize our API client
transport = RequestsHTTPTransport(url="https://api.albatros.digital/graphql", headers={})
client = Client(transport=transport, execute_timeout=30)

## Check the api version

Just to be sure that everything is set up correctly

In [42]:
from lib.queries import version

query = gql(version)
result = client.execute(query)
print("version:", result["version"])

version: 1.2.0


# Login to obtain a JWT token

With the login endpoint and a username and password, a JWT token can be obtained. The access token is valid for 24 hours, after that a new token has to be obtained.

In [43]:
from lib.queries import login

username="client"
password="secret"

query=gql(login)
params = {
    "username": username, 
    "password": password
}

result = client.execute(query, variable_values=params)

# basic error handling
if result['login']['__typename'] == "LoginError":
    print(result['login']['message'])
else:
    token = result['login']['token']['accessToken']
    transport.headers["Authorization"] = f"Bearer {token}"

Invalid credentials


# Digital Ships

The following section provides a quick run through the available

### First, let's see which Digital Ships are available

In [30]:
query = gql("""
    query digital_ship($limit: Int, $offset: Int) {
        digitalShip {
            list(limit: $limit, offset: $offset) {
                meta {
                    count
                    limit
                    offset
                }
                data {
                    id
                    name
                    modelType {
                        readableName
                    }
                }
            }
        } 
    }
""")

result = client.execute(query, variable_values={"limit": 10, "offset": 0})

print(type(result))
pprint(result['digitalShip']['list'])

<class 'dict'>
{'data': [{'id': 'bf61635a-9700-48dc-98c2-2509f389a234',
           'modelType': {'readableName': 'Trial'},
           'name': 'SARC demo'},
          {'id': '29e5d494-c1fd-4661-bc59-9d40e98153e7',
           'modelType': {'readableName': 'Trial'},
           'name': 'BULKER HANDYMAX TRIAL'},
          {'id': 'c87a4caa-5cdb-4cdb-af32-ba710c2fa8b6',
           'modelType': {'readableName': 'Type Ship'},
           'name': 'CONTAINER PANAMAX'},
          {'id': '977242a2-4536-4a5d-8f2d-351ba07371f4',
           'modelType': {'readableName': 'Type Ship'},
           'name': 'BULKER HANDYMAX2'}],
 'meta': {'count': 4, 'limit': 10, 'offset': 0}}


### Now, let's have a closer look at one of the models. For the `id` variable, choose one of the vessel id's from the previous result.

(Hint: for performance purposes, it might be wise to pick one of the models with modelType=`Trial`)

In [31]:
vessel_id = "29e5d494-c1fd-4661-bc59-9d40e98153e7"

query = gql("""
    query digital_ship($id: String!) {
        digitalShip {
            get(id: $id) {
                modelType {
                    readableName
                }
                id
                status
                name
                shipData {
                    shipType
                    lengthOverall
                    beam
                    depth
                    drafts
                    csr
                    deadweight
                    grossTonnage
                    typeOfFuel
                    speedAtCsr
                }
                outputVariables
                company {
                    name
                }
                drafts {
                    name
                    draft
                    loadcaseCount
                    failureCount
                }
            }
        } 
    }
""")

result = client.execute(query, variable_values={"id": vessel_id})

print(type(result))
pprint(result['digitalShip']['get'])

<class 'dict'>
{'company': {'name': 'AlbatrosDigital'},
 'drafts': [{'draft': 7.0,
             'failureCount': 0,
             'loadcaseCount': 384,
             'name': 'ballast_draft'},
            {'draft': 10.1,
             'failureCount': 0,
             'loadcaseCount': 384,
             'name': 'design_draft'},
            {'draft': 10.6,
             'failureCount': 0,
             'loadcaseCount': 384,
             'name': 'scantling_draft'}],
 'id': '29e5d494-c1fd-4661-bc59-9d40e98153e7',
 'modelType': {'readableName': 'Trial'},
 'name': 'BULKER HANDYMAX TRIAL',
 'outputVariables': ['Draft [m]',
                     'Speed [m/s]',
                     'TWS [m/s]',
                     'TWA [deg]',
                     'Wave direction [deg]',
                     'Wave height Hs [m]',
                     'FC ME [ton/day]',
                     'Heel [deg]',
                     'Leeway [deg]',
                     'Rudder angle [deg]',
                     'Power brake [kW]

As you can see, there is already a significant amount of data available on the Ship. This is still not a complete set of the available fields. For that, the introspection endpoint (or the graphiql dev environment) can be queried.
It's worth noting that all these fields are also available on the list endpoint from the previous cell, they are just not queried to keep the amount of returned data acceptable.

### FuelTable

From here, let's have a look at the actuel model, or, as it is called in the api: the fuel table. Note that the headers of the fueltable are identical to the `outputVariables` field in the cell above. These are all the variables that are available for this model.

In [40]:
# input parameters
vessel_id = "29e5d494-c1fd-4661-bc59-9d40e98153e7"

# optional values, can be used to filter the fuel table
draft = None
speed = None
tws = None
twa = None
waveDir = None
sigWaveHeight = None

query = gql("""
    query fuelTable($id: String!, $draft: String, $speed: Float, $tws: Float, $twa: Float, $waveDir: Float, $sigWaveHeight: Float) {
        digitalShip {
            get(id: $id) {
                fuelTable(draft: $draft, speed: $speed, tws: $tws, twa: $twa, waveDir: $waveDir, sigWaveHeight: $sigWaveHeight)
            }
        }
    }
""")

result = client.execute(query, variable_values={
    "id": vessel_id,
    "draft": draft,
    "speed": speed,
    "tws": tws,
    "twa": twa,
    "waveDir": waveDir,
    "sigWaveHeight": sigWaveHeight
})

fuel_table = result['digitalShip']['get']['fuelTable']
fuel_table_df = pd.DataFrame(fuel_table)

print(fuel_table_df.columns)

Index(['Draft [m]', 'Speed [m/s]', 'TWS [m/s]', 'TWA [deg]',
       'Wave direction [deg]', 'Wave height Hs [m]', 'FC ME [ton/day]',
       'Heel [deg]', 'Leeway [deg]', 'Rudder angle [deg]', 'Power brake [kW]'],
      dtype='object')


Now to the content of the fueltable. 

In [39]:
print(fuel_table_df)

      Draft [m]  Speed [m/s]  TWS [m/s]  TWA [deg]  Wave direction [deg]  \
0          10.6          4.5        0.0        0.0                   0.0   
1          10.6          4.5        0.0        0.0                  60.0   
2          10.6          4.5        0.0        0.0                 120.0   
3          10.6          4.5        0.0        0.0                 180.0   
4          10.6          4.5        0.0        0.0                   0.0   
...         ...          ...        ...        ...                   ...   
1147        7.0          6.0       15.0      180.0                 180.0   
1148        7.0          6.0       15.0      180.0                   0.0   
1149        7.0          6.0       15.0      180.0                  60.0   
1150        7.0          6.0       15.0      180.0                 120.0   
1151        7.0          6.0       15.0      180.0                 180.0   

      Wave height Hs [m]  FC ME [ton/day]  Heel [deg]  Leeway [deg]  \
0               

A fuel table of a Digital Ship contains a complete matrix of drafts, ship speeds, wind directions, wind speeds, significant wave heigts and wave directions. The grid size in each of these variables - and hence the total amount of rows - is determined by the model type. 

At the time of writing, the following model types exist:
- Trial: 1151 rows
- Type Ship: ~200k rows
- IMO-referenced Ship: ~200k rows